In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from numcodecs import Blosc
import os
import numpy as np
from poisson_numcodecs.Poisson import Poisson
from poisson_numcodecs import estimate
import zarr


In [ ]:
data = np.load('../tests/data/data1.npz')['data']

In [ ]:
# for bright images use count_weight_gamma=1
# for photon-starved images with few cells use count_weight_gamma = 0.001
# Otherwise, use the default

qs = estimate.compute_sensitivity(
    data.transpose(1,2,0),   # time must be in the third dimensions
    count_weight_gamma=1)  

In [ ]:
qs

In [ ]:
intensity = np.r_[qs['min_intensity']:qs['max_intensity']]
sensitivity = qs['sensitivity']
zero_level = qs['zero_level']
plt.scatter(intensity, qs['variance'], s=1)
plt.plot(intensity, (intensity - zero_level) * sensitivity, 'r')
plt.grid(True)
plt.xlabel('intensity')
plt.ylabel('variance')

In [ ]:
# make compression lookup tables
zero = np.int16(np.round(qs['zero_level']))
LUT1 = estimate.make_anscombe_lookup(sensitivity)
LUT2 = estimate.make_inverse_lookup(LUT1)

In [ ]:
# save compressed video as .gif
compressed = estimate.lookup(data - zero, LUT1)
gif_path = 'test.gif'
scale = 255//np.max(compressed)  # this makes the gif brighter. Use scale=1 normally
estimate.save_movie(compressed, gif_path, scale=scale)          
print(f'Compression ratio: {np.prod(data.shape)*2 / os.path.getsize(gif_path):0.2f}')

In [ ]:
# instantiate Poisson object
poisson_filter = Poisson(zero_level, sensitivity)

In [ ]:
# using default Zarr compressor
img = zarr.array(data, filters=[poisson_filter], compressor=Blosc(cname='zstd', clevel=1))
img.info

In [ ]:
img = zarr.open('im.zarr')

In [ ]:
plt.imshow(img[10,:,:])

In [ ]:
ls -l